In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # library for visualization
import seaborn as sns # library for visualization
import warnings
from tqdm import tqdm
import time
import os

warnings.filterwarnings('ignore')
# pd.set_option('display.max_columns', None)
sns.set_palette('Spectral')
sns.set_context('notebook', font_scale=1)
sns.set_style('whitegrid')

# Commits Status

- commit 1, date 15.11.23 "first commint"
- commit 2, date 15.11.23 "menambahkan latar belakang"
- commit 3, date 15.11.23 "coba untuk tambahkan feature harga beli mobil menggunakan dataset other. namun hasilnya belum memuaskan, masih terlalu banyak yang kosong"

# Latar Belakang

Belakangan ini, kantor pemerintahan mengadakan anggaran untuk penyewaan kendaraan roda 4. Hal tersebut membuat pihak penyedia perlu untuk menhitung profitable dalam usaha ini. Mereka perlu mendapatkan gambaran harga jual mobil bekas ketika 5 tahun kemudian. Sebagai Data Saintis kita perlu membuat model prediksi yang akurat untuk membantu keputusan investasi kendaraan roda 4.

# ETL 

## E : Extracting Data

In [2]:
# read train and test data
train=pd.read_csv('./datasets/train data/train-data.csv', index_col='Unnamed: 0')
test=pd.read_csv('./datasets/test data/test-data.csv', index_col='Unnamed: 0')
df_cars=pd.read_csv('./datasets/others data/car_data.csv', index_col='Unnamed: 0')
df_cars.reset_index(drop=True, inplace=True)
df_cars=df_cars[df_cars.Price!='Not Priced']
df_cars=df_cars[df_cars.Year>2000]

## T : Transforming Data

In [3]:
train.drop(columns=['New_Price'], inplace=True)
test.drop(columns=['New_Price'], inplace=True)

In [4]:
# transforming train cols name
new_cols=[]
for x in train.columns:
    name='_'.join(x.lower().split())
    new_cols.append(name)
    
train.columns=new_cols

In [5]:
# transforming test cols name
new_cols=[]
for x in test.columns:
    name='_'.join(x.lower().split())
    new_cols.append(name)
    
test.columns=new_cols

In [6]:
# transforming test cols name
new_cols=[]
for x in df_cars.columns:
    name='_'.join(x.lower().split())
    new_cols.append(name)
    
df_cars.columns=new_cols

In [7]:
# transforming dataset cars
df_cars['price']=df_cars.price.apply(lambda x: ''.join(x.split('$')[-1].split(',')))
df_cars['price']=df_cars.price.astype(float)
df_cars=df_cars.groupby(['model'])[['price']].mean().reset_index().sort_values('price', ascending=False)
df_cars.reset_index(drop=True, inplace=True)

In [8]:
train['status']='train'
test['status']='test'

# create new dataframe as df are concat from train dataframe and testing dataframe
df=pd.concat([train, test], axis=0)
df=df.rename(columns={'price':'price_second'})
df['text']=df.year.apply(lambda x: str(x) + ' ') + df.name.apply(lambda x: str(x))

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
# Data SKU (Product Catalog)
df_sku = df_cars.copy()

# Data Transaksi POS
df_pos = df[['text']].copy()

# Membangun model TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_sku['model'])

# Menghitung cosine similarity antara nama produk dari transaksi POS dan SKU produk
cosine_similarities = cosine_similarity(tfidf_vectorizer.transform(df_pos['text']), tfidf_matrix)

# Menentukan threshold similarity score untuk mengidentifikasi produk baru
threshold = 0.94  # Sesuaikan dengan threshold yang sesuai

# Mendapatkan SKU terbaik untuk setiap produk POS
best_sku_indexes = cosine_similarities.argmax(axis=1)
best_similarity_scores = cosine_similarities.max(axis=1)

# Menambahkan SKU, Brand, dan Type ke dataframe transaksi POS
df_pos['price'] = df_sku['price'].iloc[best_sku_indexes].values
df_pos['predict'] = df_sku['model'].iloc[best_sku_indexes].values

# Menandai produk baru jika similarity score di bawah threshold
df_pos['new_product'] = best_similarity_scores < threshold

df_pos_1=df_pos[df_pos['new_product'] == False]
df_pos_2=df_pos[df_pos['new_product'] == True]
df_pos_2['price']=np.NaN

df_pos=pd.concat([df_pos_1, df_pos_2], axis=0)
df_pos=df_pos[['new_product','text','predict','price']]
df_pos=df_pos.rename(columns={'predict':'product_similarity'})

In [11]:
df=pd.merge(df, df_pos[['text','price']], on='text')
df=df[['name','location','year','kilometers_driven','fuel_type','transmission','owner_type','mileage','engine','power','price','price_second','status']]

df['price']=df.price * 15501
df.price.fillna(0, inplace=True)
df['price']=df['price'].astype(int)

df['price_second']=df.price_second * 1000000
df.price_second.fillna(0, inplace=True)
df['price_second']=df['price_second'].astype(int)

In [12]:
train=df[df.status=='train']
test=df[df.status=='test']

In [18]:
def get_zero_value(x):
    if x == 0:
        msg=np.NaN
    else:
        msg=x
    return msg

In [20]:
train['price']=train['price'].apply(get_zero_value)
train['price_second']=train['price_second'].apply(get_zero_value)

test['price']=test['price'].apply(get_zero_value)
test['price_second']=test['price_second'].apply(get_zero_value)

In [26]:
(train.isna().sum() / len(train)) * 100

name                  0.000000
location              0.000000
year                  0.000000
kilometers_driven     0.000000
fuel_type             0.000000
transmission          0.000000
owner_type            0.000000
mileage               0.011695
engine                0.315771
power                 0.315771
price                99.514648
price_second          0.000000
status                0.000000
dtype: float64

In [27]:
(test.isna().sum() / len(test)) * 100

name                   0.000000
location               0.000000
year                   0.000000
kilometers_driven      0.000000
fuel_type              0.000000
transmission           0.000000
owner_type             0.000000
mileage                0.000000
engine                 0.432526
power                  0.432526
price                 99.106113
price_second         100.000000
status                 0.000000
dtype: float64

## L : Loading Data